In [1]:
import time
import pygame
import tkinter as tk
from tkinter import font as tkFont
from PIL import Image, ImageTk
import random
import serial
import pywitmotion as wit
import time
import threading
import csv
from datetime import datetime

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
on_serial_communication=True
connected = False
port = 'COM3'
#port = '/dev/cu.HC-06'
baud = 115200
t=5
csv_data = []
decode_q=[]
msgs_num = 0

# 비프음과 텍스트 순서대로 정의
sound_texts = [
    "앞니 바깥면으로 이동해주세요.",
    "앞니 바깥면",
    "왼쪽 바깥면",
    "오른쪽 바깥면",
    "왼쪽 아래 씹는면",
    "왼쪽 위 씹는면",
    "오른쪽 아래 씹는면",
    "오른쪽 위 씹는면",
    "앞니 위 안쪽면",
    "앞니 아래 안쪽면",
    "왼쪽 위 안쪽면",
    "왼쪽 아래 안쪽면",
    "오른쪽 위 안쪽면",
    "오른쪽 아래 안쪽면",
    "종료"
]

sound_texts2 = [
    "자유롭게 양치해주세요",
    "종료"
]

# Pygame 초기화
pygame.mixer.init()

# Set the background color to #292E3B
root = tk.Tk()
root.configure(bg="#F0F0F0")
root.title("2021013218 김도윤")

main_bg="#F0F0F0"

# Load the custom font
font_title = tkFont.Font(family="NEXON Football Gothic B", size=70)

# 프레임 생성
title_frame = tk.Frame(root, bg=main_bg)
title_frame.pack(side="top")

# 첫 번째 레이블 - "T"
title_t_label = tk.Label(title_frame, text="\nT", font=font_title, anchor="center", bg=main_bg, fg="#A89755")
title_t_label.pack(side="left")

# 두 번째 레이블 - "romatz"
title_label = tk.Label(title_frame, text="\nromatz", font=font_title, anchor="center", bg=main_bg, fg="black")
title_label.pack(side="left")

# Load the custom font
font_subtitle = tkFont.Font(family="NEXON Lv1 Gothic", size=30)

# 텍스트 표시 레이블
subtitle_label = tk.Label(root, text="김도윤\n권민욱 임대연", font=font_subtitle, anchor="center", bg=main_bg, fg="#807F83")
subtitle_label.pack(side="top", fill="both", expand=True)

# Load the custom font
font_15size = tkFont.Font(family="NEXON Football Gothic B", size=15)
font_30size = tkFont.Font(family="NEXON Football Gothic B", size=30)
font_70size = tkFont.Font(family="NEXON Football Gothic B", size=70)

# 텍스트 표시 레이블
press_kr_label = tk.Label(root, text="\n시작하려면 스페이스키를 누르세요", font=font_15size, anchor="center", bg=main_bg, fg="#479C2F")
press_kr_label.pack(side="top", fill="both", expand=True)

# 텍스트 표시 레이블
text_label = tk.Label(root, text="PRESS SPACE KEY", font=font_30size, anchor="center", bg=main_bg, fg="#479C2F")
text_label.pack(side="top", fill="both", expand=True)

# "다음구역:" 레이블
font_next = tkFont.Font(family="NEXON Football Gothic B", size=30)
next_label = tk.Label(root, text="", font=font_next, foreground="gray", bg=main_bg)

# "남은 시간:" 레이블
font_timer = tkFont.Font(family="NEXON Football Gothic B", size=50)
timer_label = tk.Label(root, text="", font=font_timer, anchor="center", bg=main_bg, fg="white")

# 창 크기 설정
root.geometry("800x400")
root.update_idletasks()
width = root.winfo_width()
height = root.winfo_height()
x = (root.winfo_screenwidth() // 2) - (width // 2)
y = (root.winfo_screenheight() // 2) - (height // 2)
root.geometry('{}x{}+{}+{}'.format(width, height, x, y))

current_text_index = 0  # 현재 텍스트의 인덱스
timer_running = False
blinking = True
is_c=False
is_space=False
msg=[]
msg_label=[]
label=None

def extract_values(s):
    values = s.strip('[]').split('],[')
    values = [val.split() for val in values]
    return [item.strip() for sublist in values for item in sublist]

def save_csv(csv_data):
    # 현재 날짜 및 시간 가져오기
    now = datetime.now()
    formatted_date_time = now.strftime("%Y%m%d_%H%M%S")

    # CSV 파일에 데이터 쓰기
    file_name = f"{formatted_date_time}.csv"
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ax', 'ay', 'az', 'gyro_x', 'gyro_y', 'gyro_z', 'label'])  # 헤더 작성
        writer.writerows(csv_data)

def save_values():
    global csv_data
    idx=0
    while idx+1<len(msg):
        # q = wit.get_quaternion(s)
        # q = wit.get_magnetic(msg)
        # q = wit.get_angle(msg)

        q1 = wit.get_acceleration(msg[idx])
        q2 = wit.get_gyro(msg[idx+1])
        msg_idx=msg_label[idx]
        print(idx,q1,q2)
        
        if (q1 is not None)and(q2 is not None):
            decode_q.append(f'{q1},{q2},{msg_idx}')
        
        idx+=1

    # Extracting and restructuring the data
    for item in decode_q:
        values = extract_values(item)
        ax, ay, az, gyro_x, gyro_y, gyro_z,label = values
        csv_data.append([ax, ay, az, gyro_x, gyro_y, gyro_z,label])

    save_csv(csv_data)

def serial_communication():
    global msg, msg_label
    if on_serial_communication==True:
        print('success')
        with serial.Serial(port, baud, timeout=5) as ser:
            s = ser.read()
            while True:
                start = time.time()
                s = ser.read_until(b'U')
                msg.append(s)
                msg_label.append(f'[{label}]')

serial_thread = threading.Thread(target=serial_communication)
serial_thread.start()

def toggle_text_label():
    if blinking:
        current_color = text_label['fg']
        target_color = "#479C2F" if current_color == main_bg else main_bg

        r1, g1, b1 = int(current_color[1:3], 16), int(current_color[3:5], 16), int(current_color[5:7], 16)
        r2, g2, b2 = int(target_color[1:3], 16), int(target_color[3:5], 16), int(target_color[5:7], 16)

        r_step = (r2 - r1) / 200  # 10 단계로 나누어서 점진적으로 변경
        g_step = (g2 - g1) / 200
        b_step = (b2 - b1) / 200

        def update_color(step):
            if blinking:
                if step <= 200:
                    r = int(r1 + r_step * step)
                    g = int(g1 + g_step * step)
                    b = int(b1 + b_step * step)
                    color = f'#{r:02X}{g:02X}{b:02X}'
                    text_label.config(fg=color)
                    press_kr_label.config(fg=color)
                    root.after(5, lambda: update_color(step + 1))
                else:
                    root.after(5, toggle_text_label)

        update_color(1)

toggle_text_label()  # 깜박임 시작

def free_brushing():
    global current_text_index, timer_running, blinking
    try:
        if blinking==False:
            text = sound_texts2[current_text_index]
            if text == "종료":
                text_label.config(text="\n"+text, fg="#BDE563", bg="#292E3B")
                timer_label.pack_forget()
                save_values()
            else:
                text_label.config(text="\n"+text, fg="#70F076", bg="#292E3B")
                # 타이머 설정
                for seconds in range(60, 0, -1):
                    if timer_running:
                        if seconds>10:
                            timer_label.config(text=f"남은 시간: {seconds} 초", bg="#292E3B", fg="white")
                        else:
                            timer_label.config(text=f"남은 시간: {seconds} 초", bg="#292E3B", fg="red")
                        root.update()
                        time.sleep(1)
                    else:
                        break  # 중지 요청이 들어오면 타이머 종료
                if timer_running:
                    print(current_text_index)
                    root.after(0, free_brushing)
                    pygame.mixer.music.load("beep.wav")
                    pygame.mixer.music.play()
                else:
                    timer_label.config(text="타이머 중지", bg="#292E3B")
            current_text_index += 1
    except IndexError:
        text_label.config(text="종료, 감사합니다", fg="#70F076")

def area_brushing():
    global current_text_index, timer_running, blinking,label
    try:
        if blinking==False:
            text = sound_texts[current_text_index]
            if text == "종료":
                save_values()
                text_label.config("\n"+text, fg="#BDE563", bg="#292E3B")
                next_label.pack_forget()
                timer_label.pack_forget()
            else:
                # print('ss')
                print(is_space)
                label=current_text_index-1
                next_text_index = current_text_index + 1
                next_text = sound_texts[next_text_index]
                next_label.pack()
                timer_label.pack()
                text_label.config(text="\n"+text, fg="#70F076", bg="#292E3B")
                next_label.config(text=f"\n다음구역: {next_text}", bg="#292E3B")
                # 타이머 설정
                if current_text_index==0:
                    for seconds in range(1, 0, -1):
                        if timer_running:
                            if seconds>=4:
                                timer_label.config(text=f"남은 시간: {seconds} 초", bg="#292E3B", fg="white")
                            else:
                                timer_label.config(text=f"남은 시간: {seconds} 초", bg="#292E3B", fg="red")
                            root.update()
                            time.sleep(1)
                        else:
                            break  # 중지 요청이 들어오면 타이머 종료
                else:
                    for seconds in range(10, 0, -1):
                        if timer_running:
                            if seconds>=4:
                                timer_label.config(text=f"남은 시간: {seconds} 초", bg="#292E3B", fg="white")
                            else:
                                timer_label.config(text=f"남은 시간: {seconds} 초", bg="#292E3B", fg="red")
                            root.update()
                            time.sleep(1)
                        else:
                            break  # 중지 요청이 들어오면 타이머 종료
                if timer_running:
                    print(current_text_index)
                    pygame.mixer.music.load("beep.wav")
                    pygame.mixer.music.play()
                    next_label.pack_forget()
                    timer_label.pack_forget()
                    text_label.config(text=f"\n다음구역: {next_text}", fg="#70F076", bg="#292E3B") 
                    root.after(1500, area_brushing)
                else:
                    timer_label.config(text="타이머 중지", bg="#292E3B")
            current_text_index = next_text_index
    except IndexError:
        text_label.config(text="종료, 감사합니다", fg="#70F076")

def c_sequence():
    global current_text_index, timer_running, csv_data, decode_q
    current_text_index = 0
    decode_q=[]
    timer_running = True  # 타이머 시작
    # Start the sequence
    free_brushing()

# Create a function to start the sequence
def space_sequence():
    global current_text_index, timer_running, csv_data, decode_q
    current_text_index = 0
    timer_running = True  # 타이머 시작
    decode_q=[]
    # Start the sequence
    area_brushing()

def on_c_key(event):
    global timer_running, blinking,is_c
    print(is_space)
    if is_space!=True:
        root.configure(bg="#292E3B")
        text_label.config(fg="#70F076",font=font_70size)

        if timer_running:
            is_c=False
            timer_running = False  # 타이머 중지
            if(timer_label.winfo_ismapped):
                root.configure(bg="#F0F0F0")
                text_label.pack_forget()
                timer_label.pack_forget()
                title_frame.pack()
                subtitle_label.pack()
                press_kr_label.pack()
                text_label.pack()
                text_label.config(text="PRESS SPACE KEY", font=font_30size, anchor="center", bg=main_bg, fg="#479C2F")
                blinking = True
                toggle_text_label()
        else:
            is_c=True
            blinking = False
            title_frame.pack_forget()
            subtitle_label.pack_forget()
            press_kr_label.pack_forget()
            text_label.pack()
            timer_label.pack()
            c_sequence()

def on_space(event):
    print(is_c)
    global timer_running, blinking,is_space,msg,msg_label
    if is_c!=True:
        root.configure(bg="#292E3B")
        text_label.config(fg="#70F076",font=font_70size)

        if timer_running:
            is_space=False
            timer_running = False  # 타이머 중지
            if(next_label.winfo_ismapped):
                root.configure(bg="#F0F0F0")
                text_label.pack_forget()
                next_label.pack_forget()
                timer_label.pack_forget()
                title_frame.pack()
                subtitle_label.pack()
                press_kr_label.pack()
                text_label.pack()
                text_label.config(text="PRESS SPACE KEY", font=font_30size, anchor="center", bg=main_bg, fg="#479C2F")
                blinking = True
                toggle_text_label()
        else:
            is_space=True
            blinking = False
            title_frame.pack_forget()
            subtitle_label.pack_forget()
            press_kr_label.pack_forget()
            text_label.pack()
            next_label.pack()
            timer_label.pack()
            msg=[]
            msg_label=[]
            space_sequence()

root.bind("<space>", on_space)
root.bind("<c>", on_c_key)

root.mainloop()


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/serial/serialposix.py", line 322, in open
    self.fd = os.open(self.portstr, os.O_RDWR | os.O_NOCTTY | os.O_NONBLOCK)
FileNotFoundError: [Errno 2] No such file or directory: 'COM3'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 910, in run
    self._target(*self._args, **self._kwargs)
  File "/var/folders/pj/5xhy0s2d6bx728d6w_6bwp1w0000gn/T/ipykernel_6375/1119308603.py", line 153, in serial_communication
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/serial/serialutil.py", line 244, in __init__
    self.open()
  File "/Li

success


2024-01-30 16:59:34.285 Python[6375:80764] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


False
True
0
True


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/tkinter/__init__.py", line 1892, in __call__
    return self.func(*args)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/tkinter/__init__.py", line 814, in callit
    func(*args)
  File "/var/folders/pj/5xhy0s2d6bx728d6w_6bwp1w0000gn/T/ipykernel_6375/1119308603.py", line 264, in area_brushing
    timer_label.config(text=f"남은 시간: {seconds} 초", bg="#292E3B", fg="red")
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/tkinter/__init__.py", line 1646, in configure
    return self._configure('configure', cnf, kw)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/tkinter/__init__.py", line 1636, in _configure
    self.tk.call(_flatten((self._w, cmd)) + self._options(cnf))
_tkinter.TclError: invalid command name ".!label5"


In [3]:
sound_texts = [
    "앞니 바깥면",
    "오른쪽 바깥면",
    "왼쪽 바깥면",
    "오른쪽 아래 씹는면",
    "오른쪽 위 씹는면",
    "왼쪽 아래 씹는면",
    "왼쪽 위 씹는면",
    "오른쪽 위 안쪽면",
    "오른쪽 아래 안쪽면",
    "왼쪽 위 안쪽면",
    "왼쪽 아래 안쪽면",
    "앞니 위 안쪽면",
    "앞니 아래 안쪽면",
    "종료"
]